In [32]:
import pandas as pd
import numpy as np
# filter warnings
import warnings
warnings.filterwarnings("ignore")
from  sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# Executing datamerge function, to create final dataset
%run ./datamerge.py

In [37]:
final_telco_df =mergeData() 

In [4]:
final_telco_df.head()

,Customer ID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Satisfaction Score,Customer Status,Churn Label,Churn Value,Churn Score,CLTV,Churn Category,Churn Reason,ID,Population
0,8779-QRDMV,1,United States,California,Los Angeles,90022,"34.02381, -118.156582",34.02381,-118.156582,Male,...,3,Churned,Yes,1,91,5433,Competitor,Competitor offered more data,21,68701
1,4737-AQCPU,1,United States,California,Los Angeles,90022,"34.02381, -118.156582",34.02381,-118.156582,Male,...,3,Stayed,No,0,42,4658,NaN,NaN,21,68701
2,5043-TRZWM,1,United States,California,Los Angeles,90022,"34.02381, -118.156582",34.02381,-118.156582,Female,...,5,Joined,No,0,34,3503,NaN,NaN,21,68701
3,8165-CBKXO,1,United States,California,Los Angeles,90022,"34.02381, -118.156582",34.02381,-118.156582,Male,...,3,Stayed,No,0,46,5748,NaN,NaN,21,68701
4,9979-RGMZT,1,United States,California,Los Angeles,90022,"34.02381, -118.156582",34.02381,-118.156582,Female,...,3,Stayed,No,0,38,5545,NaN,NaN,21,68701


In [38]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(final_telco_df.loc[:, final_telco_df.columns != 'Churn Label']
                                                , final_telco_df[['Churn Label']], test_size=0.3, random_state=42,validation)

In [40]:
Ytrain.columns

Index(['Churn Label'], dtype='object')

# Feature engineering and processing on Train data

In [41]:
# Since Count, Country ,State and Quarter are constant, we are dropping them from further analysis. 
Xtrain = Xtrain.drop(['Count','Country','State','Quarter'],axis=1)

In [42]:
Xtrain.columns

Index(['Customer ID', 'City', 'Zip Code', 'Lat Long', 'Latitude', 'Longitude',
       'Gender', 'Age', 'Under 30', 'Senior Citizen', 'Married', 'Dependents',
       'Number of Dependents', 'Referred a Friend', 'Number of Referrals',
       'Tenure in Months', 'Offer', 'Phone Service',
       'Avg Monthly Long Distance Charges', 'Multiple Lines',
       'Internet Service', 'Internet Type', 'Avg Monthly GB Download',
       'Online Security', 'Online Backup', 'Device Protection Plan',
       'Premium Tech Support', 'Streaming TV', 'Streaming Movies',
       'Streaming Music', 'Unlimited Data', 'Contract', 'Paperless Billing',
       'Payment Method', 'Monthly Charge', 'Total Charges', 'Total Refunds',
       'Total Extra Data Charges', 'Total Long Distance Charges',
       'Total Revenue', 'Satisfaction Score', 'Customer Status', 'Churn Value',
       'Churn Score', 'CLTV', 'Churn Category', 'Churn Reason', 'ID',
       'Population'],
      dtype='object')

In [43]:
# Customer Demographics
# Extracting FSA Code from ZIP Code - first 3 digits of Zip Code
# Reducing the dimensionality of City with 1100 + values
Xtrain['FSA'] = Xtrain['Zip Code'].astype(str).str[:3]

In [45]:
Xtrain['FSA'].head()

1695    952
1095    923
3889    906
3667    953
2902    904
Name: FSA, dtype: object

In [46]:
len(final_telco_df['City'].unique())

1106

In [51]:
len(final_telco_df['Zip Code'].unique())

1626

In [41]:
len(final_telco_df['FSA'].unique())

57

In [47]:
firstq = Xtrain.groupby('FSA')['Population'].mean().describe()['25%']
meanpop = Xtrain.groupby('FSA')['Population'].mean().describe()['mean']
thirdq = Xtrain.groupby('FSA')['Population'].mean().describe()['75%']

In [48]:
Xtrain['RegionType']= Xtrain['Population'].apply(lambda x: 'Small Urban' if x < firstq
                                                                 else ('Medium Sized Urban' if (x >= firstq and x  < meanpop)else('Metropolitan' if (x >= meanpop and x  < thirdq) else 'Large Metropolitan')))                                                                                                                             

In [49]:
Xtrain.groupby('RegionType')['Customer ID'].count()

RegionType
Large Metropolitan    1401
Medium Sized Urban     664
Metropolitan           627
Small Urban           2238
Name: Customer ID, dtype: int64

# Part Worths of Charges for each utility

In [50]:
# Total Charges here refer to Monthly Charge * Tenure in Months
Xtrain['Total_Charges_Allservices']  = (Xtrain['Monthly Charge']*Xtrain['Tenure in Months'])+Xtrain['Total Long Distance Charges']+Xtrain['Total Extra Data Charges']
Xtrain['Perc_Services'] = Xtrain['Total Charges']/Xtrain['Total_Charges_Allservices']
Xtrain['Perc_LongDistance'] =Xtrain['Total Long Distance Charges']/Xtrain['Total_Charges_Allservices']
Xtrain['Perc_AdditionalData'] =Xtrain['Total Extra Data Charges']/Xtrain['Total_Charges_Allservices']
Xtrain[['Perc_Services','Perc_LongDistance','Perc_AdditionalData']].head()

,Perc_Services,Perc_LongDistance,Perc_AdditionalData
1695,0.983795,0.000000,0.031838
1095,0.374380,0.628317,0.000000
3889,0.775832,0.219363,0.010469
3667,0.934303,0.068222,0.000000
2902,0.739461,0.237453,0.000000


# Engagement Features

In [51]:
Xtrain[['Phone Service','Online Security', 'Online Backup', 'Device Protection Plan','Streaming TV', 'Streaming Movies',
       'Streaming Music', 'Unlimited Data','Internet Service','Multiple Lines','Premium Tech Support']].head()

,Phone Service,Online Security,Online Backup,Device Protection Plan,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Internet Service,Multiple Lines,Premium Tech Support
1695,No,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes
1095,Yes,No,No,No,No,No,No,No,No,No,No
3889,Yes,No,Yes,Yes,No,Yes,Yes,No,Yes,Yes,No
3667,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
2902,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes


In [52]:
names = ['Phone Service','Online Security', 'Online Backup', 'Device Protection Plan','Streaming TV', 'Streaming Movies',
       'Streaming Music', 'Unlimited Data','Internet Service','Multiple Lines','Premium Tech Support']
def ConvertValues(names,df):
    for i in names:
        df[i] = np.where(df[i] =='Yes',1,0)   
ConvertValues(names,Xtrain)

In [53]:
Xtrain[['Phone Service','Online Security', 'Online Backup', 'Device Protection Plan','Streaming TV', 'Streaming Movies',
       'Streaming Music', 'Unlimited Data','Internet Service','Multiple Lines','Premium Tech Support']].head()

,Phone Service,Online Security,Online Backup,Device Protection Plan,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Internet Service,Multiple Lines,Premium Tech Support
1695,0,0,0,1,1,1,1,0,1,0,1
1095,1,0,0,0,0,0,0,0,0,0,0
3889,1,0,1,1,0,1,1,0,1,1,0
3667,1,0,1,1,1,1,1,1,1,1,1
2902,1,1,1,1,1,1,0,1,1,1,1


In [54]:
Xtrain['No_of_Phone_Services'] = Xtrain['Phone Service']+Xtrain['Multiple Lines']
Xtrain['No_of_Security_Services'] = Xtrain['Online Security']+Xtrain['Online Backup']+Xtrain['Device Protection Plan']+Xtrain['Premium Tech Support']
Xtrain['No_of_Media_Services'] = Xtrain['Streaming TV']+Xtrain['Streaming Movies'] + Xtrain['Streaming Music']
Xtrain['No_of_Internet_Services'] = Xtrain['Unlimited Data']+Xtrain['Internet Service']

In [55]:
Xtrain[['No_of_Phone_Services','No_of_Security_Services','No_of_Media_Services','No_of_Internet_Services']].head()

,No_of_Phone_Services,No_of_Security_Services,No_of_Media_Services,No_of_Internet_Services
1695,0,2,3,1
1095,1,0,0,0
3889,2,2,2,1
3667,2,3,3,2
2902,2,4,2,2


# Binning Age Variable

Creating Additional Age bins

Under 30 - Already exists

30 to 45 - Young family

45 to 60 - Grown family

more than 60 - Senior Citizen - Already exists

In [56]:
Xtrain['Young_Family'] = np.where((Xtrain['Age'] >= 30) & (Xtrain['Age'] <45),1,0)

In [57]:
Xtrain[['Age','Young_Family']].head()

,Age,Young_Family
1695,33,1
1095,59,0
3889,40,1
3667,20,0
2902,77,0


In [58]:
Xtrain['Grown_Family'] = np.where((Xtrain['Age'] >= 45) & (Xtrain['Age'] <60),1,0)

In [59]:
Xtrain[['Age','Grown_Family']].tail(20)

,Age,Grown_Family
2391,71,0
769,30,0
1685,58,1
130,77,0
2919,26,0
3171,21,0
3444,23,0
6231,53,1
5578,24,0
4426,37,0


In [60]:
Xtrain.columns

Index(['Customer ID', 'City', 'Zip Code', 'Lat Long', 'Latitude', 'Longitude',
       'Gender', 'Age', 'Under 30', 'Senior Citizen', 'Married', 'Dependents',
       'Number of Dependents', 'Referred a Friend', 'Number of Referrals',
       'Tenure in Months', 'Offer', 'Phone Service',
       'Avg Monthly Long Distance Charges', 'Multiple Lines',
       'Internet Service', 'Internet Type', 'Avg Monthly GB Download',
       'Online Security', 'Online Backup', 'Device Protection Plan',
       'Premium Tech Support', 'Streaming TV', 'Streaming Movies',
       'Streaming Music', 'Unlimited Data', 'Contract', 'Paperless Billing',
       'Payment Method', 'Monthly Charge', 'Total Charges', 'Total Refunds',
       'Total Extra Data Charges', 'Total Long Distance Charges',
       'Total Revenue', 'Satisfaction Score', 'Customer Status', 'Churn Value',
       'Churn Score', 'CLTV', 'Churn Category', 'Churn Reason', 'ID',
       'Population', 'FSA', 'RegionType', 'Total_Charges_Allservices',
    

In [64]:
Xtrain = Xtrain[['Gender','Under 30','Senior Citizen', 'Married', 'Dependents','Referred a Friend','Tenure in Months', 'Offer',
                'Avg Monthly GB Download','Internet Type','Contract','Paperless Billing','Payment Method','RegionType','Perc_Services', 
                 'Perc_LongDistance', 'Perc_AdditionalData','No_of_Phone_Services', 'No_of_Security_Services','No_of_Media_Services', 
                 'No_of_Internet_Services', 'Young_Family','Grown_Family','Satisfaction Score','Total Revenue']]

In [75]:
l1 = ['Paperless Billing','Under 30','Senior Citizen', 'Married', 'Dependents','Referred a Friend']
ConvertValues(l1,Xtrain)

In [81]:
df_cat = Xtrain[['Gender','Internet Type','Contract','Payment Method','RegionType','Offer']]

In [82]:
df_cat = pd.get_dummies(df_cat)

In [83]:
df_cat.head()

,Gender_Female,Gender_Male,Internet Type_Cable,Internet Type_DSL,Internet Type_Fiber Optic,Internet Type_None,Contract_Month-to-Month,Contract_One Year,Contract_Two Year,Payment Method_Bank Withdrawal,...,RegionType_Large Metropolitan,RegionType_Medium Sized Urban,RegionType_Metropolitan,RegionType_Small Urban,Offer_None,Offer_Offer A,Offer_Offer B,Offer_Offer C,Offer_Offer D,Offer_Offer E
1695,0,1,0,1,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
1095,0,1,0,0,0,1,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
3889,0,1,0,0,1,0,1,0,0,1,...,0,0,1,0,0,1,0,0,0,0
3667,1,0,0,0,1,0,0,1,0,1,...,0,0,0,1,0,0,1,0,0,0
2902,1,0,0,0,1,0,0,0,1,1,...,0,1,0,0,1,0,0,0,0,0


In [84]:
Xtrain1 = pd.concat([Xtrain.loc[:, ~Xtrain.columns.isin(['Gender','Internet Type','Contract','Payment Method','RegionType','Offer'])], df_cat],axis=1)

In [85]:
Xtrain1.head()

,Under 30,Senior Citizen,Married,Dependents,Referred a Friend,Tenure in Months,Avg Monthly GB Download,Paperless Billing,Perc_Services,Perc_LongDistance,...,RegionType_Large Metropolitan,RegionType_Medium Sized Urban,RegionType_Metropolitan,RegionType_Small Urban,Offer_None,Offer_Offer A,Offer_Offer B,Offer_Offer C,Offer_Offer D,Offer_Offer E
1695,0,0,0,0,0,57,28,0,0.983795,0.000000,...,0,0,0,1,1,0,0,0,0,0
1095,0,0,0,0,0,31,0,0,0.374380,0.628317,...,0,0,0,1,1,0,0,0,0,0
3889,0,0,1,0,1,71,7,1,0.775832,0.219363,...,0,0,1,0,0,1,0,0,0,0
3667,1,0,1,0,1,57,73,1,0.934303,0.068222,...,0,0,0,1,0,0,1,0,0,0
2902,0,1,1,0,1,70,25,1,0.739461,0.237453,...,0,1,0,0,1,0,0,0,0,0


# Model Building

In [86]:
model = RandomForestClassifier()
from sklearn.model_selection import cross_val_score
# m1= cross_val_score(model, Xtrain, Ytrain, cv=10,scoring='accuracy')

In [97]:
Xtrain11, Xtest11, Ytrain11, Ytest11 = train_test_split(Xtrain1,Ytrain, test_size=0.2, random_state=42)

In [98]:
model.fit(Xtrain11, Ytrain11)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [100]:
ypred = model.predict(Xtest11)

In [101]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Ytest11, ypred))

0.9665314401622718


In [103]:
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = Xtrain11.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
Satisfaction Score,0.565355
Contract_Two Year,0.046476
Tenure in Months,0.039371
Total Revenue,0.038323
Perc_Services,0.030983
Avg Monthly GB Download,0.028200
Perc_LongDistance,0.027036
Internet Type_Fiber Optic,0.025072
No_of_Media_Services,0.015777
Senior Citizen,0.015665
